In [ ]:
from imports import *
%matplotlib inline

In [ ]:
datadir = "data/phononTracks/"
dataset = datadir+"singleStep-iZIP5_51250813_0000.root"

In [ ]:
def getHits(file):
    """Load phonon collection data for plotting."""
    if not file: return None
        
    branches = ["EventNum","Track","Step","PName","ChanNum","QETNum","X1","Y1","Z1","X3","Y3","Z3"]
    hits = CDataFrame("G4SimDir/mczip0", file).AsNumpy(branches)
    hits["R1"] = np.sqrt(hits["X1"]**2+hits["Y1"]**2)
    hits["R3"] = np.sqrt(hits["X3"]**2+hits["Y3"]**2)
    return hits

In [ ]:
hits = getHits(dataset)

In [ ]:
plt.hist((hits["R3"][abs(hits["Z3"])>0.012699]*1e3)**2,bins=100)
plt.title("Phonon hit positions, excluding sidewalls")
plt.xlabel("R3^2 [mm^2]")
plt.savefig(datadir+"Hits_vs_Rsq.png")

In [ ]:
chanPA = hits["ChanNum"]==0
chanPB = hits["ChanNum"]==1
chanTop = (hits["ChanNum"]>=0) & (hits["ChanNum"]<4)
hasQET = hits["QETNum"]>=0
PAZ = []
PAqetZ = []
PBZ = []
PBqetZ = []
PtopZ = []
PqetZ = []

Z = np.arange(-12,13)
for z in Z:
    zcut = ((z-0.5)<hits["Z1"]*1e3) & (hits["Z1"]*1e3<(z+0.5))
    PAZ.append(np.count_nonzero(chanPA & zcut))
    PAqetZ.append(np.count_nonzero(chanPA & hasQET & zcut))
    PBZ.append(np.count_nonzero(chanPB & zcut))
    PBqetZ.append(np.count_nonzero(chanPB & hasQET & zcut))
    PtopZ.append(np.count_nonzero(chanTop & zcut))
    PqetZ.append(np.count_nonzero(chanTop & hasQET))

PAZ = np.array(PAZ)
PAqetZ = np.array(PAqetZ)
PBZ = np.array(PBZ)
PBqetZ = np.array(PBqetZ)
PtopZ = np.array(PtopZ)
PqetZ = np.array(PqetZ)

In [ ]:
plt.plot(Z, PAZ/PtopZ, label="PA/Ptop")
plt.plot(Z, PBZ/PtopZ, label="PB/Ptop")
plt.title("Channel area fractions vs. Z1 at sidewall")
plt.xlabel("Z1 [mm]")
plt.ylabel("Channel fraction, 1 mm bins")
plt.legend()
plt.savefig(datadir+"ChannelFrac_vs_Z1.png")

In [ ]:
plt.plot(Z, PAqetZ/PqetZ, label="PA/Ptop")
plt.plot(Z, PBqetZ/PqetZ, label="PB/Ptop")
plt.title("Channel QET fractions vs. Z1 at sidewall")
plt.xlabel("Z1 [mm]")
plt.ylabel("Channel fraction, 1 mm bins")
plt.legend()
plt.savefig(datadir+"ChanQETfrac_vs_Z1.png")

In [ ]:
plt.plot(Z, PAqetZ/PAZ, label="PA")
plt.plot(Z, PBqetZ/PBZ, label="PB")
plt.title("QET coverage vs. Z1 at sidewall")
plt.xlabel("Z1 [mm]")
plt.ylabel("Collection QET/area, 1 mm bins")
plt.legend()
plt.savefig(datadir+"ChanQETarea_vs_Z1.png")

In [ ]:
ycut = abs(hits["Y1"]*1e3)<0.5
xzcut = (abs(hits["X1"]*1e3-37.7444)<0.2) & (abs(hits["Z1"])*1e3<0.2)
topcut = abs(hits["Z3"])*1e3>12.65
print((ycut&xzcut&topcut).sum())

In [ ]:
plt.scatter(hits["X3"][ycut&xzcut&topcut],hits["Y3"][ycut&xzcut&topcut])
plt.savefig(datadir+"Z0-Y0_topCaustics.png")